In [9]:
import pandas as pd
from IPython.display import display

pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [10]:
all_ohlcv = pd.read_csv('data/ohlcv.csv.gz', compression='gzip')

In [11]:
all_ohlcv.head(10)

,timestamp,symbol,open,high,low,close,volume
0,2024-01-01 00:00:00,BTC/USDT,42283.5800,42554.5700,42261.0200,42475.2300,1271.6811
1,2024-01-01 00:00:00,DOGE/USDT,0.0896,0.0900,0.0895,0.0898,17799677.0000
2,2024-01-01 00:00:00,XRP/USDT,0.6155,0.6172,0.6146,0.6162,14498728.0000
3,2024-01-01 00:00:00,ETH/USDT,2281.8700,2297.1800,2281.2700,2295.5100,10771.9183
4,2024-01-01 00:00:00,SOL/USDT,101.7200,102.7900,101.5600,101.9600,196680.9300
5,2024-01-01 01:00:00,BTC/USDT,42475.2300,42775.0000,42431.6500,42613.5600,1196.3786
6,2024-01-01 01:00:00,DOGE/USDT,0.0898,0.0903,0.0895,0.0902,29985288.0000
7,2024-01-01 01:00:00,XRP/USDT,0.6161,0.6190,0.6150,0.6185,9029149.0000
8,2024-01-01 01:00:00,SOL/USDT,101.9600,104.7500,101.5100,104.1200,242095.3300
9,2024-01-01 01:00:00,ETH/USDT,2295.5200,2306.6000,2292.9000,2303.7200,8413.4260


In [20]:
data = all_ohlcv.copy().sort_values(by=['symbol', 'timestamp'])

window = 14

delta = data['close'] - data['open']
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)

avg_gain = gain.groupby(data['symbol']).transform(
    lambda x: x.ewm(com=window - 1, min_periods=window, adjust=False).mean()
)
avg_loss = loss.groupby(data['symbol']).transform(
    lambda x: x.ewm(com=window - 1, min_periods=window, adjust=False).mean()
)

relative_strength = avg_gain / avg_loss

rsi = 100 - (100 / (1 + relative_strength))

# Append the calculated RSI to the DataFrame
data['rsi'] = rsi

display(data.head(100))

,timestamp,symbol,open,high,low,close,volume,rsi
0,2024-01-01 00:00:00,BTC/USDT,42283.5800,42554.5700,42261.0200,42475.2300,1271.6811,NaN
5,2024-01-01 01:00:00,BTC/USDT,42475.2300,42775.0000,42431.6500,42613.5600,1196.3786,NaN
13,2024-01-01 02:00:00,BTC/USDT,42613.5700,42638.4100,42500.0000,42581.1000,685.2198,NaN
18,2024-01-01 03:00:00,BTC/USDT,42581.0900,42586.6400,42230.0800,42330.4900,794.8039,NaN
24,2024-01-01 04:00:00,BTC/USDT,42330.5000,42399.9900,42209.4600,42399.9900,715.4176,NaN
...,...,...,...,...,...,...,...,...
476,2024-01-04 23:00:00,BTC/USDT,44318.0100,44355.2200,44050.0000,44151.1000,1308.4734,59.5545
480,2024-01-05 00:00:00,BTC/USDT,44151.1000,44248.4700,43950.8400,44211.1000,1764.7743,60.5768
486,2024-01-05 01:00:00,BTC/USDT,44211.1000,44284.0000,42450.0000,43376.0000,5302.0483,43.9318
494,2024-01-05 02:00:00,BTC/USDT,43376.0000,43972.8600,43302.0000,43506.0000,3111.0085,46.4009
